In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# %config Completer.use_jedi = False

import os
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
import optuna
import optiver2023
import random
from tqdm import tqdm
random.seed(100)

optiver2023.make_env.func_dict['__called__'] = False
env = optiver2023.make_env()
iter_test = env.iter_test()

REDUCED = True


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

print('python:', sys.version)

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

python: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


In [15]:
train = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
test = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv')
sample_submission = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv')
revealed_targets = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv')

In [16]:
train.head(5)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [17]:
test.tail(5)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33000 entries, 0 to 32999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   stock_id                 33000 non-null  int64  
 1   date_id                  33000 non-null  int64  
 2   seconds_in_bucket        33000 non-null  int64  
 3   imbalance_size           33000 non-null  float64
 4   imbalance_buy_sell_flag  33000 non-null  int64  
 5   reference_price          33000 non-null  float64
 6   matched_size             33000 non-null  float64
 7   far_price                14771 non-null  float64
 8   near_price               15000 non-null  float64
 9   bid_price                33000 non-null  float64
 10  bid_size                 33000 non-null  float64
 11  ask_price                33000 non-null  float64
 12  ask_size                 33000 non-null  float64
 13  wap                      33000 non-null  float64
 14  time_id               

In [18]:
revealed_targets.head(5)
revealed_targets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33162 entries, 0 to 33161
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   stock_id           33000 non-null  float64
 1   date_id            33162 non-null  int64  
 2   seconds_in_bucket  33162 non-null  int64  
 3   time_id            33162 non-null  int64  
 4   revealed_target    33000 non-null  float64
 5   revealed_date_id   33000 non-null  float64
 6   revealed_time_id   33000 non-null  float64
dtypes: float64(4), int64(3)
memory usage: 1.8 MB


In [19]:
sample_submission.head(5)

,time_id,row_id,target
0,26290,478_0_0,0
1,26290,478_0_1,0
2,26290,478_0_2,0
3,26290,478_0_3,0
4,26290,478_0_4,0


In [20]:
# distribution except for stock_id, date_id, seconds_in_bucket, row_id
# fig = plt.figure()
# ax = fig.add_subplot(1,1,1)
train = train.drop(columns=['stock_id','row_id','seconds_in_bucket','row_id', 'far_price', 'near_price'])
val = test.drop(columns=['stock_id','row_id','seconds_in_bucket','row_id', 'far_price', 'near_price', 'currently_scored'])
print(val)
# boxplot_labels = plt_train.columns
# sns.boxplot(train['imbalance_size'])

       date_id  imbalance_size  imbalance_buy_sell_flag  reference_price  \
0          478      3753451.43                       -1         0.999875   
1          478       985977.11                       -1         1.000245   
2          478       599128.74                        1         1.000584   
3          478      2872317.54                       -1         0.999802   
4          478       740059.14                       -1         0.999886   
...        ...             ...                      ...              ...   
32995      480      2440722.89                       -1         1.000317   
32996      480       349510.47                       -1         1.000643   
32997      480            0.00                        0         0.995789   
32998      480      1000898.84                        1         0.999210   
32999      480      1884285.71                       -1         1.002129   

       matched_size  bid_price   bid_size  ask_price   ask_size       wap  \
0       11

In [21]:
# 欠損値がどのくらいあるか見てみる
# 今回のデータは幅が広そうなので中央値でnull埋め
# すべて量的データ

null_data = train.isnull()

print('data size:', len(train))
print('null sum', null_data.sum())
print('===============================')

if REDUCED: train = train[:1000] # データ量が多すぎるのでいったん1000件くらいで

data size: 5237980
null sum date_id                      0
imbalance_size             220
imbalance_buy_sell_flag      0
reference_price            220
matched_size               220
bid_price                  220
bid_size                     0
ask_price                  220
ask_size                     0
wap                        220
target                      88
time_id                      0
dtype: int64


In [22]:
fillna_train = pd.DataFrame()
col_names = train.columns
for col_name in col_names:
    fillna_train[col_name] = train[col_name].fillna(train[col_name].median()) #いったん中央値で試す

print(fillna_train.isnull().sum())
print('columns length ->', len(col_names))

date_id                    0
imbalance_size             0
imbalance_buy_sell_flag    0
reference_price            0
matched_size               0
bid_price                  0
bid_size                   0
ask_price                  0
ask_size                   0
wap                        0
target                     0
time_id                    0
dtype: int64
columns length -> 12


In [23]:
# モデル、学習用関数の定義

EPOCHS = 5
BATCH_SIZE = 32
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

class Dataset:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# NNとLightGBMは分離させる
class NeuralNetEnsembleModel(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.nn_params = params

        # layer
        self.input_layer = nn.Linear(self.nn_params['input_dim'], self.nn_params['embed_dim'])
#         self.attn_layer = nn.MultiheadAttention(embed_dim=self.nn_params['embed_dim'], 
#                                                 num_heads=self.nn_params['num_heads'], 
#                                                 batch_first=True)
        self.output_layer = nn.Linear(self.nn_params['embed_dim'], self.nn_params['output_dim'])
    
    def forward(self, X):      
        y = self.input_layer(X)
#         y = self.attn_layer(y)
        y = self.output_layer(y)
        return y
                 
        # resnetを入れる？

# NNの学習
def train(model, dataloader, optimizer, loss_fn, device):
    model.train() #学習モード
    train_loss = 0

    for batch_X, batch_y in tqdm(dataloader): # ミニバッチ学習
        batch_X, batch_y = batch_X.to(device), batch_y.to(device) # gpuに送る
        optimizer.zero_grad()
        pred = model.forward(batch_X)
        pred = pred[:, 0]
        
        loss = loss_fn(pred, batch_y)
        loss.backward()
        optimizer.step() # ネットワーク更新
        train_loss += loss.item()

    return train_loss / len(dataloader)

def eval(model, dataloader, optimizer, loss_fn, device):
    model.eval() #評価モード
    eval_loss = 0
    
    for batch_X, batch_y in tqdm(dataloader):
        with torch.no_grad():
            batch_X, batch_y = batch_X.to(device), batch_y.to(device) # gpuに送る
            pred = model.forward(batch_X)
            pred = pred[:, 0]

    #         print(pred.shape)
    #         print(batch_y.shape)

            loss = loss_fn(pred, batch_y)
    #         print('val loss', loss.item())
            eval_loss += loss.item()
    return eval_loss / len(dataloader) # lossの合計が大きくなりすぎてオーバーフロー。nan
    


In [24]:
# LightGBM学習実行

train_X = fillna_train.drop('target', axis=1)
train_y = fillna_train['target']

test_X = val
test_y = revealed_targets['revealed_target']

# lgbm_model = lgbm.LGBMRegressor(**params['lgbm_params'])
# lgbm_model.fit(train_X, train_y)
# lgbm.plot_importance(lgbm_model, importance_type="gain")


In [25]:
# params
col_names = train_X.columns
params = {
    'lgbm_params':{
        'objective': 'regression',
        'metrics': 'mae',
        'n_estimators': 500,
        'random_state': 100,
        'learning_rate': 0.01,
        'device': 'gpu'
    },
    'nn_params':{
        'input_dim': len(col_names),
        'embed_dim': len(col_names)*2,
        'num_heads': len(col_names),
        'output_dim': 1,
        'learning_rate': 0.01
    }
}

In [26]:
# NN学習
# NNに突っ込むためにはテーブルデータをベクトルに変換

train_X = train_X.values
train_y = train_y.values
test_X = test_X.values
test_y = test_y.values

train_X = torch.from_numpy(train_X.astype(np.float32)).clone()
train_y = torch.from_numpy(train_y.astype(np.float32)).clone()
test_X = torch.from_numpy(test_X.astype(np.float32)).clone()
test_y = torch.from_numpy(test_y.astype(np.float32)).clone()



print('train_X', train_X.shape)
print('train_y', train_y.shape)
print('eval_X', test_X.shape) 
print('eval_y', test_y.shape) 

model = NeuralNetEnsembleModel(params=params['nn_params']).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.L1Loss()

train_dataset = Dataset(train_X, train_y)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataset = Dataset(test_X, test_y)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

train_losses = []
test_losses = []
for epoch in range(EPOCHS):
    
    train_loss = train(model=model, dataloader=train_dataloader, 
                            optimizer=optimizer, loss_fn=loss_fn, device=device)
    train_losses.append(train_loss)
    
    test_loss = eval(model=model, dataloader=test_dataloader, 
                          optimizer=optimizer, loss_fn=loss_fn, device=device)
    test_losses.append(test_loss)
    
#     print(f'epoch:{epoch}, train loss:{train_loss}, val loss:{test_loss}')
    print(f'epoch:{epoch}, train loss:{train_loss}')

print('********************************************************************')
print(f'average loss -> train:{sum(train_losses)/len(train_losses)}')
print(f'average loss -> train:{sum(train_losses)/len(train_losses)}, val:{sum(test_losses)/len(test_losses)}')

train_X torch.Size([1000, 11])
train_y torch.Size([1000])
eval_X torch.Size([33000, 11])
eval_y torch.Size([33162])


100%|██████████| 1032/1032 [00:00<00:00, 1752.37it/s]


epoch:0, train loss:355656.2861328125


100%|██████████| 1032/1032 [00:00<00:00, 1751.08it/s]


epoch:1, train loss:172228.193359375


...

epoch:2, train loss:145732.76831054688


100%|██████████| 1032/1032 [00:00<00:00, 1694.81it/s]


epoch:3, train loss:170956.72625732422


100%|██████████| 1032/1032 [00:00<00:00, 1745.78it/s]

epoch:4, train loss:108271.59619140625
********************************************************************
average loss -> train:190569.11405029296
average loss -> train:190569.11405029296, val:nan


In [27]:
# 提出
counter = 0
device = torch.device('cpu')
model.to(device)
model.eval()
for (test, revealed_targets, prediction) in iter_test:
    if counter == 0:
        print(test.head(3))
        print(revealed_targets.head(3))
        print(prediction.head(3))
        print(len(prediction))
    head_idx = counter * 200
    pred = model.forward(test_X)[:, 0].to('cpu').detach().numpy().copy()
#     print('prediction', pred)
    prediction['target'] = pred[head_idx: head_idx+200]
#     print('****************')
    env.predict(prediction)
#     print(prediction.tail(3))
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
   stock_id  date_id  seconds_in_bucket  imbalance_size  \
0         0      478                  0      3753451.43   
1         1      478                  0       985977.11   
2         2      478                  0       599128.74   

   imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                       -1         0.999875   11548975.43        NaN   
1                       -1         1.000245    3850033.97        NaN   
2                        1         1.000584    4359198.25        NaN   

   near_price  bid_price  bid_size  ask_price  ask_size  wap   row_id  \
0         NaN   0.999875  22940.00   1.000050   9177.60  1.0  478_0_0   
1         NaN   0.999940   1967.90   1.000601  19692.00  1.0  478_0_1   
2         NaN   0.999918   4488.22   1.000636  34955.12  1.0  478_0_2   

   currently_scored  
0             False  
1           